# Importa dependências

In [17]:
import pdfplumber as pdfpb
import json
import os
import re
import os.path as osp
import pathlib
import tarfile
import bz2
from zipfile import ZipFile
from pyunpack import Archive

In [47]:
compressed_files = [
    #'288_licitacoes-pirapetinga.tar.xz',
    #'289-licitacoes-coqueiral.zip',
    #'290-licitacoes-cristais.zip',
    #'302-licitacoes-olaria.7z',
    #'304-licitacoes-passa-vinte.tar.gz',
    #'306-licitacoes-arantina.7z',
    #'353-licitacoes-ijaci.7z',
    #'381-licitacoes-sao-bento-abade.zip',
    '385-ribeirao_vermelho.zip'

]

In [44]:
# funções úteis
def list_by_file_format(path, file_format):
    path = path.rstrip("/") + "/"
    file_descriptor = [json.loads(line) for line in open(path + "file_description.jsonl").readlines()]
    files = [item["file_name"] for item in file_descriptor if item["type"] in file_format]
    return [f"{path}{f}".rstrip(";") for f in files]

def list_pdf(path):
    return list_by_file_format(path, ["application/pdf", "pdf", "pdf;"])

def list_csv(path):
    return list_by_file_format(path, ".csv")

def beauty_print(grouped):
    for city, processes in grouped.items():
        print(city + ":")
        total = 0
        for no_process, documents in processes.items():
            total += len(documents)
            print(f"\t{no_process:>8s}: {len(documents)}")
        print(f"\t{'total':>8s}: {total}")

def decompress_file(tmp_path, full_filepath, filename, extension):
    '''
    Decompress a file in a specified directory.
    '''
    
    if extension == '.zip':
        file_obj = ZipFile(full_filepath, 'r')
    elif tarfile.is_tarfile(full_filepath):
        file_obj = tarfile.open(full_filepath, 'r')
    elif extension == '.7z':
        file_obj = Archive(full_filepath)
    elif not extension:
        print('Nothing to do, file uncompressed')
        return None
    
    
    try:
        file_obj.extractall(tmp_path)
    except(FileNotFoundError):
        print("AQUI")
        pass

In [48]:

'''aux_path = "/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/documentos_municipios/"



# Decompressing
for file in compressed_files:
    print(file)
    tmp_path = os.path.join(base_path, str(file).split(".")[0])
    main_path = os.path.join(aux_path, file)
    extension = pathlib.Path(main_path).suffix
    print(extension)
    decompress_file(tmp_path, main_path, file, extension)
    tmp_path = base_path
    main_path = aux_path
    
print("Arquivos descomprimidos...")'''

385-ribeirao_vermelho.zip
.zip
Arquivos descomprimidos...


In [51]:
# list path
#base_path = "../data/documentos_municipios/docs-cidades-jul2021/"
# list path
base_path = "../data/documentos_municipios/docs-cidades-jul2021/"
pattern = "\\d+[-_]licitacoes-(([a-z0-9]|-)+)"

sel_path = os.listdir(base_path)
sel_path = list(filter(lambda f: re.match(pattern, f), sel_path)) # Seleciona os que iniciam com <cod>-licitacoes-<cidade>
cities = [re.match(pattern, p)[1] for p in sel_path if not p.endswith("bh")] # Remove BH da lista

# arquivos por cidade
files = {city: list_pdf(f"{base_path}{p}/data/files/")  for city, p in zip(cities, sel_path)}
print("cities: " + ", ".join(files.keys()))
print([(key, len(docs)) for key, docs in files.items()])

cities: pirapetinga, cristais, coqueiral, olaria, arantina, passa-vinte, ijaci, sao-bento-abade, itamarati2017, itamarati2018, itamarati2019, itamarati2020, ribeirao-vermelho
[('pirapetinga', 1007), ('cristais', 1736), ('coqueiral', 1528), ('olaria', 42), ('arantina', 983), ('passa-vinte', 395), ('ijaci', 451), ('sao-bento-abade', 232), ('itamarati2017', 301), ('itamarati2018', 317), ('itamarati2019', 384), ('itamarati2020', 108), ('ribeirao-vermelho', 684)]


# Converte Formatos: PDF → JSON

In [53]:
def exception_wrap(*, callback, default):
    if not callable(callback):
        raise Error("@func must be a callable.")
    try:
        return callback()
    except:
        return default
    
def convert(*, filename):
    result = {
        "file_id": filename.rsplit("/")[-1].rsplit(".")[0], # foo/bar/baz.any -> baz.any -> baz
        "original_name": filename.rsplit("/")[-1], 
        "n_pages": -1, 
        "text_content": "", 
        "table_content": [], 
        "status": "FAILED"
    }
    try:
        with pdfpb.open(filename) as f:
            extract_text_func = lambda pages: [page.dedupe_chars().extract_text() for page in pages]
            extract_table_func = lambda pages: [page.extract_table() for page in pages]
        
            result["n_pages"] = len(f.pages)
            
            result["text_content"] = exception_wrap(callback=lambda:extract_text_func(f.pages), default=[])
            result["text_content"] = [text for text in result["text_content"] if text]
            
            result["table_content"] = exception_wrap(callback=lambda:extract_table_func(f.pages), default=[])            
            result["table_content"] = [table for table in result["table_content"] if table]
            
            if bool(result["text_content"] or result["table_content"]): # empty list is cast to false
                result["status"] = "SUCCESS"
    except:
        pass

    return result

In [ ]:
for city, documents in files.items():
    print(city)
    filename = f"{base_path}documentos_json/{city}.json"        
    with open(filename, "wt", encoding="utf-8") as f:
        for document in documents:
            print(document)
            content = convert(filename=document)
            obj = json.dumps(content)
            f.write(obj + "\n") # Salva cada documento em uma linha
        

pirapetinga
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/bd9cf430f53cfa701f8c7d51e2e0ba2f
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/bc0866e315473780ab8d83b14278998f
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/c7bd8928fa0ff4856199e1eb2589e8dc
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/25bff9c51ff33ab73b3ec2a5c1ae8ea7
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/5df4600b8c2d6bfc9fbdb9034a937966
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/511789d507851ccf6d17b40a7c7e7187
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/524080521adefcb1f54fc1411d8b4fe9
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/4b3fd4d29753a7d8da07d36b853e9010
../data/docu

../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/9c2a74e1640e46d1a0a7f8f159f07701
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/60b68dc388c5112f7dd2fb8daa540877
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/25448a3bff6cef22b1c98302bea34feb
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/2cd4d24a8087d350570f331482b501bc
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/5694a3e26f516e4ec2b49b11ce80f4a2
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/8f199a31d003fa8a175ff15c940fffba
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/d1abf86fc023f35f44b2ef779bc1f715
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/a25ee5e1077ac0b84eda9df1409106a0
../data/documentos_munic

../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/feb9d6a07db4036331378670574e22d0
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/7789412e8e9f1e63c6c902550c4c67ac
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/c695a1bc3cb8f143b77e49a50a847ea4
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/10db253c16fb24ba5224c2c61b416fba
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/580f7a5da1a3027d972cdfe724124dfc
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/543240247bd7ea0b935d087d6f8d4ab4
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/1f918b48ee37d84e27e31458f9c81737
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/4876c5b8ec6b2d2f447d8e6415e73daf
../data/documentos_munic

../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/17c6050f6b17da3ea969166b834af301
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/6840a80be091da4bff81105d86ce4bed
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/77304dc31d4af4824aeabe530819ea10
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/ffa51f164da65551614459cdbc1b2ea7
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/51833660952918ce46dbde52b975da06
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/e5573f53f18a593a3e32c79b8b22d815
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/76ff5603cc037152d9cfa7bf9c4f0515
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/f56882135a2fa259dbb0dc043bb7e06c
../data/documentos_munic

../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/47c4f19d5e9b4cff4c71b46cae8a8b16
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/8d0d23c3a3a3c5dc6190aecc6772c4b4
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/bfd6b0ab430801e76605fba0108df416
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/c97a17fdc713270f6d258e95e88e382e
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/6c3479e0b8d036e024c25d00e871c192
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/b8369183cf1c0fe00a9668c967ee2995
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/fdd06fe65781ad68a5c5b4e03b44ceac
../data/documentos_municipios/docs-cidades-jul2021/288_licitacoes-pirapetinga/data/files/a10fdeee5b68184745b7b48d206dd8c1
../data/documentos_munic

...

../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/286d2ee887e3f8ab9a3e1f3424e3e820.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/b65dae066077af352a6d6581602c73a0.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/fb3005ed731327a10982b969c97bc38f.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/f10c43613bf38ffdeed0e5ca6c7b8321.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/b60481ef7d2b0cc15c9568b6ed694d91.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/35a994611efd8f9655259fecd663ae6b.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/092ab9329c18646cdc65010aaf6c5f3b.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/778a56762d78fc44bc5a7d6164de7f61.pdf
../data/document

../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/f51b10469c989355877a81addcc0514e.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/317e82a7c4018b4c5152ad89e1f98a54.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/6d639b681a38e5e59f6cde64f73e658e.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/4db603016cce6fdb159ef2b18413a88f.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/e15df70bde0bfed9d04b3d37565202e2.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/6a6227bef96142da31be7d2d0737db9c.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/eba287089e28eae50a7329b5f392b275.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/4926233eefe604b196265e9cefdcaa26.pdf
../data/document

../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/77858a9945a4f221b2089854f55ce24f.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/168567f2e86a427f3a830653c5603b4e.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/14aefa648253cd55f590ec2d2880d1f8.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/6ee2883fe8585b89ffa61d962b421c08.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/3eb9f5113deb08e95a7a7a69992567a2.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/bd6dec37d929c78e2abe302bf386cb4e.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/aa73f90823f560473100d59c1dbc217c.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/f8836becb948ff4a16f399bd7a64fd73.pdf
../data/document

../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/98884a94841e9e971ec41af690ef7387.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/681dd15604431c61d7348a0b921710e1.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/800d107ba3ac1924829e6ef7cc9c69b7.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/79e718c4725754bd0f18a17c3e19ee2d.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/46e64b68380304d548ad2256146d361b.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/c9a7754c0b827dcfd8f1ac24dbab9cf6.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/7f28dc1e5762ce9f51b98a1d92bbf474.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/31cc9104ee01758c871b7bb67c7baca3.pdf
../data/document

../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/d08d18740ab8fd1b820e9652128eafb2.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/b20b1e33996fe15040b6365f62809a44.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/fd9508e5dd7a4aba9df107a1bf8b1f50.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/827a398f255b73d083d073b4e3aed5cf.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/3d15830fcebf10277f94e7b91267d863.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/84cb6bac8f4ad8557c2599dcf0eb216d.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/6c23ad437c4b5740b2e26fe1ee3963b1.pdf
../data/documentos_municipios/docs-cidades-jul2021/290-licitacoes-cristais/data/files/bde038d751d19e7604861d372d5ca311.pdf
../data/document

# Exemplo de uso

In [ ]:
cities

In [ ]:
filename = f"{base_path}documentos_json/pirapetinga.json"
with open(filename) as f:
    lines = f.readlines() # lê o conteúdo (pode ser lido em um stream, se achar necessário)
    print("document id                     ", "  #", " status")
    for line in lines: # um doc por linha
        document = json.loads(line) # alguns arquivos tem problemas, portanto, verifique o status.
        print(document["file_id"], f"{document['n_pages']:>03d}", document["status"])
        print(document["text_content"])